In [ ]:
# try:
#     from mc_lib.rndm import RndmWrapper
# except ImportError:
#     !pip install git+https://github.com/ev-br/mc_lib.git@master
#     pass

In [ ]:
%load_ext cython

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import cython
import numpy as np
cimport numpy as np
from libc.time cimport time
from libc.math cimport exp
from mc_lib.rndm cimport RndmWrapper
from mc_lib.lattices._common import tabulate_neighbors

#from mc_lib.observable cimport RealObservable

@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray initState(long[::1] lattice,
                                    RndmWrapper rndm):
    """
        :param   lattice: unconfigured array of future lattice (empty array)
        :param   rndm: class with random methods from mc_lib
        :return: change the 'Lattice' variable and return nothing
    """
    for i in range(lattice.shape[0]):
        lattice[i] = 1 if rndm.uniform() > 0.5 else -1
    return

@cython.boundscheck(False)
@cython.wraparound(False)
cdef np.ndarray mcmove(long[::1] config,
                                 double beta,
                                        long[:, ::1] ngb,
                                                     RndmWrapper rndm):
    """
        One flip attempt
        :param    config: Current configuration of lattice
        :param    beta:   Inversed temperature of current configuration
        :param    L:      Linear size 'L' of lattice
        :param    ngb:    Array of neigbours
    """
    cdef:
        Py_ssize_t site = int(config.shape[0] * rndm.uniform())
        Py_ssize_t site1 = 0
        double dE = 0
        long num_ngb = ngb[site, 0]
    for n in range(1, num_ngb + 1):
        site1 = ngb[site, n]
        dE += config[site1] * config[site]
    cdef double ratio = exp(-2 * dE * beta)
    if rndm.uniform() > ratio:
        return
    config[site] *= -1
    return

@cython.boundscheck(False)  # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire functio
cdef double calcEnergy(long[::1] config,
                                 long[:, ::1] ngb):
    """
        Count current configuration energy
        :param    config: Current configuration of lattice
        :param    ngb:    Array of neigbours
        return:
    """
    cdef:
        Py_ssize_t site = 0
        Py_ssize_t site1 = 0
        double energy = 0
    for site in range(config.shape[0]):
        num_ngb = ngb[site, 0]
        for n in range(1, num_ngb+1):
            site1 = ngb[site, n]
            energy += -1 * config[site] * config[site1]
    return energy / 4.

@cython.boundscheck(False)
@cython.wraparound(False)
def IsingSimulate(L, T, sweeps, int T_corr, int seed = np.random.randint(0, 1000), int rseed = 1234):
"""
    L - linear_size
    T - One temperature point
    Sweeps - number of L^2 Metropolis Monte-Karlo steps
    T_corr - number of algorithm steps between which snapshots are taken
"""
cdef RndmWrapper rndm = RndmWrapper((rseed, seed))
    cdef:
        float beta = 1.0 / T
        int sweep = 0
        int steps_per_sweep = L * L
        int num_therm = int(12 * L * L)
        int i = 0

    cdef:
        np.ndarray configs = np.empty([sweeps, L*L], dtype=int)
        long[:, ::1] ngb = tabulate_neighbors(L, kind='sq')
        long[::1] config = np.empty(L*L, dtype=int)
    initState(config, rndm)
    for sweep in range(num_therm):
        for i in range(steps_per_sweep):
            mcmove(config, beta, ngb, rndm)

    for sweep in range(sweeps):
        for i in range(T_corr):
            for j in range(steps_per_sweep):
                mcmove(config, beta, ngb, rndm)
        configs[sweep] = config.copy()
    return configs

### Run (Example)

In [ ]:
L = 20
sweeps = 1000
T_c = tc = 2 / (np.log(1 + 2 ** 0.5))
t_corr = {10: 150*4, 20: 150*4, 30: 150*4, 40: 310*4, 80: 580*4, 120: 3300*2}

config = ng.IsingSimulate(L=L,
                          T=T_c,
                          sweeps=sweeps,
                          T_corr = t_corr[L])